In [81]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math

In [60]:
n_candidates = 2

In [61]:
total = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

In [62]:
def countSearch(page):
    
    pattern = {'saúde' : ['saú','enferm', 'hospital',  'médicos'],
               'segurança' : ['segur',' polic','crim','violên',' lei',' Milit','ordem pública','pen',' defe',' infração','infrator','presidi','preso','vigilân','investig'],
               'educação' : ['educação',' ensino',' professor',' escolas'],
               'economia' : [' economi',' econômi',' produ',' mercado',' indústr',' comérci',' agro',' agric',' serviços',' setor',' terciário',' desenvolv',' terceiriz']}
    
    candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}
   
    
    page = requests.get(page)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for key, words in pattern.items():                          
        for w in words:
            candidate[key]+=str(soup.find_all('p')).upper().count(w.upper())
    
    return candidate

## Manuela D'Ávila (PC do B)

In [99]:
#trocar esse site pois não mostra as propostas do candidato
manuela = countSearch("http://bardebatom.com.br/noticia/manuela-davila-conheca-a-candidata-a-presidencia")
peso_manuela = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

## Levi Fidelix

In [100]:
levi = countSearch("http://congressoemfoco.uol.com.br/eleicoes-2014/programa-de-governo-de-levy-fidelix/")
peso_levi = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

## TERM FREQUENCY

In [101]:
def tf(candidate):
    
    tf_candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}
    sum_candidate = sum(candidate.values())
    
    for key, words in total.items():
        tf_candidate[key] = candidate[key]/sum_candidate
    
    return tf_candidate

In [102]:
def count(manuela, levi, key):
    cont = 0
    if(manuela[key] > 0):
        cont+=1
    if(levi[key] > 0):
        cont+=1
    return cont

In [103]:
def idf(manuela, levi):
    
    idf_candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}
    
    for key, words in total.items():
        idf_candidate[key] = np.log(n_candidates/count(manuela, levi, key))
    
    return idf_candidate

In [104]:
def tf_idf(total, candidate, manuela, levi):
    
    ans = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}
    tf1 = tf(candidate)
    idf1 = idf(manuela, levi)

    for key, words in ans.items():
        ans[key] = tf1[key] * idf1[key]
    
    return ans

In [105]:
for key, words in total.items():
    total[key] = manuela[key] + levi[key]

print(total)

{'saúde': 13, 'segurança': 28, 'educação': 4, 'economia': 82}


## Manuela 

In [108]:
manuela

{'economia': 22, 'educação': 0, 'saúde': 2, 'segurança': 11}

In [106]:
print(tf_idf(total, manuela, manuela, levi))

{'saúde': 0.0, 'segurança': 0.0, 'educação': 0.0, 'economia': 0.0}


## Levi

In [110]:
levi

{'economia': 60, 'educação': 4, 'saúde': 11, 'segurança': 17}

In [111]:
print(tf_idf(total, levi, manuela, levi))

{'saúde': 0.0, 'segurança': 0.0, 'educação': 0.030136833937388925, 'economia': 0.0}
